In [1]:
!pip install transformers
!pip install transformers[torch]
!pip install -U sentence-transformers
!pip install annoy

In [2]:
# Процесс формирования бд для поиска заготовленных ответов. Не юзать локально!
"""
from annoy import AnnoyIndex
import pandas as pd
from sentence_transformers import SentenceTransformer
import time
from tqdm import tqdm
import pickle as pkl
import torch

MODEL_NAME = 'sentence-transformers/all-MiniLM-L6-v2'
n_trees = 8096
max_corpus_size = 4000
top_k_hits = 3
annoy_index_path = 'Annoy_index'
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
torch.save(model, '/content/modelbert.pt')


def get_data():
    pd.read_excel('/content/datatender.xlsx').to_csv('QAdata', index=False)
    data = pd.read_csv('./QAdata')
    return list(data.QUESTION)


def new_data():
    pd.read_excel('/content/datatender.xlsx').to_csv('QAdata', index=False)
    data = pd.read_csv('./QAdata')
    return list(data.QUESTION)

def timing_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"{func.__name__} time: {end_time - start_time:.2f} seconds")
        return result
    return wrapper

@timing_decorator
def embedding():
    question = get_data()

    embeddings = []
    for q in tqdm(question, desc='Embedding'):
        embeddings.append(model.encode(q))
    with open('model.pkl', 'wb') as f:
        pkl.dump(embeddings, f)
    return embeddings

@timing_decorator
def build_annoy_index(embeddings):
    annoy_index = AnnoyIndex(len(embeddings[0]), 'angular')
    for i, emb in tqdm(enumerate(embeddings), desc='Building index', total=len(embeddings)):
        annoy_index.add_item(i, emb)
    annoy_index.build(n_trees)
    annoy_index.save(annoy_index_path)
    return annoy_index


embeddings = embedding()
annoy_index = build_annoy_index(embeddings)
"""

' \nfrom annoy import AnnoyIndex\nimport pandas as pd\nfrom sentence_transformers import SentenceTransformer\nimport time\nfrom tqdm import tqdm\nimport pickle as pkl\nimport torch\n\nMODEL_NAME = \'sentence-transformers/all-MiniLM-L6-v2\'\nn_trees = 8096\nmax_corpus_size = 4000\ntop_k_hits = 3\nannoy_index_path = \'Annoy_index\'\nmodel = SentenceTransformer(\'sentence-transformers/all-MiniLM-L6-v2\')\ntorch.save(model, \'/content/modelbert.pt\')\n\n\ndef get_data():\n    pd.read_excel(\'/content/datatender.xlsx\').to_csv(\'QAdata\', index=False)\n    data = pd.read_csv(\'./QAdata\')\n    return list(data.QUESTION)\n\n\ndef new_data():\n    pd.read_excel(\'/content/datatender.xlsx\').to_csv(\'QAdata\', index=False)\n    data = pd.read_csv(\'./QAdata\')\n    return list(data.QUESTION)\n\ndef timing_decorator(func):\n    def wrapper(*args, **kwargs):\n        start_time = time.time()\n        result = func(*args, **kwargs)\n        end_time = time.time()\n        print(f"{func.__name__} 

In [3]:
from annoy import AnnoyIndex
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import pickle as pkl
import torch

data = pd.read_csv('QAdata')
question = list(data.QUESTION)
answer = list(data.ANSWER)
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
with open('model.pkl', 'rb') as f:
    embeddings = pkl.load(f)
annoy_index = AnnoyIndex(len(embeddings[1]), 'angular')
annoy_index.load('Annoy_index')

def question_response(embeddings):
    top_k_hits = 5
    inp_question = input("Введите ваш вопрос: ")
    question_embedding = model.encode(inp_question)
    corpus_ids, scores = annoy_index.get_nns_by_vector(question_embedding, top_k_hits, include_distances=True)
    hits = []
    for id, score in zip(corpus_ids, scores):
        hits.append({'corpus_id': id, 'score': 1-((score**2) / 2)})
    hits_above_threshold = [hit for hit in hits if hit['score'] > 0.8]
    if hits_above_threshold:
        print("Возможные ответы на ваш вопрос:")
        for hit in hits_above_threshold:
            print("\t{:.3f}\t{}".format(hit['score'], question[hit['corpus_id']]))
        question_embedding_tensor = torch.tensor(question_embedding)
        embeddings = torch.tensor(embeddings)
        correct_hits = util.semantic_search(question_embedding, embeddings, top_k=top_k_hits)[0]
        correct_hits_ids = list([hit['corpus_id'] for hit in correct_hits])
        print("Наиболее подходящий ответ на ваш вопрос:")
        #print(answer[correct_hits_ids[0]])
        with open('user_questions.txt', 'a') as f:
            f.write(f"Вопрос пользователя: {inp_question}\n")
        return answer[correct_hits_ids[0]]
    else:
        print("К сожалению, мы не поняли ваш вопрос. Возможно, вы имели в виду один из следующих вопросов:")
        for hit in hits[:3]:
            print("\t{}".format(question[hit['corpus_id']]))
        return "Не понятен вопрос"




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
question_response(embeddings)

Введите ваш вопрос: Какие штрафы предусмотрены за неисполнение контракта в 44-ФЗ?
Возможные ответы на ваш вопрос:
	1.000	Какие штрафы предусмотрены за неисполнение контракта в 44-ФЗ?
	0.811	Должен ли заказчик вносить сведения об оплате штрафа поставщиком в ЕИС?
Наиболее подходящий ответ на ваш вопрос:


<ipython-input-3-68f179655967>:31: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  embeddings = torch.tensor(embeddings)


'Перечень штрафов приведен в Постановлении Правительства РФ от 30.08.2017 N 1042 (ред. от 02.08.2019), а пеня начисляется по нормам Гражднаского Кодекса РФ как одна трехсотая действующей на дату уплаты пени ставки рефинансирования Центрального банка Российской Федерации от цены контракта, уменьшенной на сумму, пропорциональную объему обязательств, предусмотренных контрактом и фактически исполненных поставщиком (подрядчиком, исполнителем)'